# Connection Rule
## 1. Internal Connection
- 연결이 2개 이상인 경우 wire 로 처리한다.

## 2. Inter Connection

- wire 이름은 mst 기준으로 사용한다.


mst - slv
- slv cnt_con = 1 : mst wire 이름을 slv 에 기록한다.
- slv cnt_con > 1 : slv wire 이름을 사용하고 mst와 slave 연결은 assign으로 처리한다. (3.1 예외 처리 참고)




## 3. Top Connection
top(output) - " "
- '' 포트에 대한 할당이 없음 top port 바로 연결
- top port size >= " " port : top port 에 대한 [] 처리
- top port size  < " " port : concatenation 처리, 사용하지 않는 port 는 tmp wire 처리

top(input) - " "
- '' 포트에 대한 할당이 없음 top port 바로 연결
- top port size = " " port : top port 를 바로 연결
- top port size >= " " port : top port 에 대한 [] 처리 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> assign 처리
- top port size  < " " port : concatenation 처리, 사용하지 않는 port 는 tmp wire 처리 >>> assign 처리

top(output) - "mst"
- mst cnt_con = 1 : top 을 바로 연결 >>>>>>>>>>>>>>>>>>>>>>>>>>>> concatenation 처리는 추후에
- mst cnt_con > 1 : mst wire를 이용하여 assign 연결

top(output) - "slv"
- mst cnt_con = 1 : top 을 바로 연결 >>>>>>>>>>>>>>>>>>>>>>>>>>>> concatenation 처리는 추후에
- mst cnt_con > 1 : slv wire를 이용하여 assign 연결




## 3. 예외처리

3.1 한 slv에 연결된 mst가 여러개 있을때 mst wire 이름이 충돌나게 된다. -> slv 도 wire 이름을 사용하고 assign으로 연결한다.




1. mster/slave port search
2. 해당 port의 type 설정
   - type 이 비어 있으면 mst/slv/top 설정
   - type 이 이미 설정되어 있는 경우, 기존 type과 설정하려는 type 이 다르면 Error   
     (한 port 에 2개의 type 이 공존할 수 없다)
3. master/slave 의 dir이 아래 조건을 만족하지 않으면 Error
   - master : 'output'            slave : 'input'
   - master : 'input'             slave : 'output'
   - master or slave : 'output'   top   : 'output'
   - master or slave : 'input;    top   : 'input'
4. 'wire' 설정을 한다.
   - 'wire' 이름은 master port 기준으로 선언한다. (bit size로 master port 기준)
   - 연결된 master/slave port 의 'wire' 이름은 동일하다.
   - 예외) slv 와 연결된 mst가 1나 이상일때, slv 이름을 기술하고 mst/slv 연결을 assign으로 처리
   - 'mst'는 concatenation 이 없다.
   - 'slv'는 concatenation 이 있다. (slv에 연결된 mst가 1나 이상일때 assign으로 처리)
   
5. 문서 기준   
   - 'mst' - 'slv' : 'mst' 기준으로 wire 선언
   - 'mst' - 'top' : 'mst' 기준으로 wire 선언 top 연결은 assign 처리
   - 'top' 이 input인 경우 top port 이름으로 연결한다.
   - 'top' - 'mst' : 'mst'가 이미 선언되어 있는 경우, top은 assign 으로 처리한다.
   - 'top' - '' : 
   - 'top'(input) - 'mst' : 'top' port 이름으로 연결

   * top이 연결은 마지막에
* internal connection 을 먼저 처리한후 top 을 처리 한다. (top output 연결시 1
   - slave의 'wire' 이름은 여러개 일 수 있다.
   
- master - slave port 둘다 같은 이름으로 'wire' 선언을 한다..
   - master - slave port 둘 중에 하나가 'wire' 선언이 되어 있는경우, 선언된 이름을 사용한다.
   - master or slave - top(output) 기존에 선언된 'wire'가 있으면, assign 선언한다.
   - master or slave - top(output) 기존에 선언된 'wire'가 없으면, output port name을 'wire'로 사용한다.
   - master or slave - top(input) 경우 input port 를 'wire' 로 선언한다.
                                              



## Module Function

gen_port

gen_module

## Hierachy Function

insert_top

insert_inst



# RTL Generation
### Connection Function

gen_conb



# Example

### Port Generation
port_list = []
port_list.append(gen_port('A', 'output', 0, 0))

### module Generation
dic_mst = gen_module ('master', 'master.v', mst_port_list)
dic_slv = gen_module ('slave', 'slave.v', slv_port_list)
dic_top = gen_module ('top', 'top.v', top_port_list)

### Hierarchy Generation
dic_result = {}
dic_result = insert_top(dic_result, dic_top)
dic_result = insert_inst(dic_result, dic_mst, 'u_mst')
dic_result = insert_inst(dic_result, dic_slv, 'u_slv')


--------------------------------

### TOP JSON 을 Port 기준으로 Dictionary (SYS_CON) 를 만든다 

### CON JSON 을 정보를 SYS_CON에 추가한다.
con_insert(SYS_CON, 'u_mst.A', 'u_slv.M')

con_insert(SYS_CON, 'u_mst.A', 'u_slv.N')





```
{inst_name : {port_name : {
                   ccnt:            // connection count
                   type:            // mst, slv, top
                   dir:             // input, output, inout
                   msb :
                   lsb : 
                   wire : 
```